In [83]:
# store start time to get execution time of entire script
import time
start_time = time.time()

In [84]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import csv

df = pd.read_csv('TX_ENGAGEMENT_IMP.csv')
print(df.shape)
df.head()

(23961, 108)


,ID,female,nonwhite,unemplmt,primsev,Raceg4,B2a_0,State,City,zipcode,noins,prsatx,tottxp4,TRI_0,TRIg_0,TMI_0,TMIg_0,SES_0,SESg_0,RFQ33c,PMS20,PMS20g,phpi18,phpig,GVS,gvsg,tsd_0,und15,CWSg_0,srprobg,S9y10,dldiag,press,DSS9_0,dssg_0,epsg_0,adhdg_0,cdsg_0,suicprbs_0,cjsig_0,lrig_0,srig_0,homeless_0,HsgStress_0,S6,PSSI_0,pssig_0,OSSI_0,ossig_0,SSEI_0,SSEIg_0,GSSI_0,gssig_0,SSSI_0,sssig_0,GSI_0,gsig_0,GCS_0,gcsg_0,rerig_0,ncar,LRI7_0,SRI7_0,loc,txtypeg,ada_0,engage30,engage42,init,female_cd,nonwhite_cd,unemplmt_cd,primsev_cd,Raceg4_cd,prsatx_cd,TRI_0_cd,TRIg_0_cd,TMI_0_cd,TMIg_0_cd,SES_0_cd,SESg_0_cd,GVS_cd,gvsg_cd,tsd_0_cd,und15_cd,CWSg_0_cd,srprobg_cd,S9y10_cd,dldiag_cd,DSS9_0_cd,dssg_0_cd,epsg_0_cd,adhdg_0_cd,cdsg_0_cd,suicprbs_0_cd,cjsig_0_cd,lrig_0_cd,srig_0_cd,homeless_0_cd,S6_cd,GCS_0_cd,gcsg_0_cd,ncar_cd,LRI7_0_cd,SRI7_0_cd,loc_cd,txtypeg_cd,ada_0_cd
0,3819,0.0,0.0,1.0,2.0,1.0,11,AZ,Tucson,85716,NaN,1.0,10,0.0,0.0,1.0,1.0,5.0,0.0,20.0,NaN,0.0,NaN,NaN,4.0,2.0,1.0,1.0,0.0,1.0,2.0,1.0,NaN,6.0,2.0,1.0,2.0,1.0,0.0,1.0,1.0,1.0,0.0,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,8.0,9.0,1.0,2.0,90.0,1,1,1,0,0,1,2,1,1,0,0,1,1,5,0,4,2,1,1,0,1,2,1,6,2,1,2,1,0,1,1,1,0,0,0,0,0,8,9,1,2,90
1,1908,0.0,0.0,0.0,1.0,1.0,13,IA,Iowa City,52242-5005,NaN,0.0,10,0.0,0.0,1.0,1.0,5.0,0.0,3.0,NaN,2.0,NaN,NaN,1.0,1.0,0.0,1.0,0.0,0.0,8.0,0.0,NaN,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0,0.0,NaN,NaN,NaN,NaN,6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,10.0,8.0,1.0,7.0,86.0,0,0,0,0,0,0,1,1,0,0,0,1,1,5,0,1,1,0,1,0,0,8,0,1,0,1,1,1,0,1,1,1,0,0,0,0,1,10,8,1,7,86
2,951,0.0,0.0,0.0,3.0,1.0,13,OH,Cleveland,44113-1512,NaN,0.0,8,1.0,1.0,0.0,2.0,5.0,0.0,19.0,14.0,0.0,NaN,NaN,0.0,0.0,0.0,1.0,0.0,1.0,8.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,0.0,0.0,11.0,1.0,7.0,6.0,0,1,1,0,0,0,3,1,0,1,1,0,2,5,0,0,0,0,1,0,1,8,1,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,11,1,7,6
3,1032,0.0,0.0,0.0,3.0,1.0,13,OH,Cleveland,44113-1512,NaN,0.0,12,1.0,1.0,1.0,1.0,5.0,0.0,5.0,5.0,1.0,NaN,NaN,4.0,2.0,0.0,1.0,0.0,1.0,8.0,1.0,NaN,2.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,0.0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,NaN,1.0,14.0,10.0,1.0,7.0,69.0,1,1,1,0,0,0,3,1,0,1,1,1,1,5,0,4,2,0,1,0,1,8,1,2,1,1,1,1,0,0,2,1,0,1,3,1,1,14,10,1,7,69
4,1039,0.0,0.0,0.0,2.0,1.0,13,OH,Cleveland,44113-1512,NaN,0.0,11,2.0,1.0,1.0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,8.0,0.0,NaN,2.0,1.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,13.0,11.0,1.0,7.0,74.0,1,1,1,0,0,0,2,1,0,2,1,1,1,5,0,0,0,0,1,0,0,8,0,2,1,0,0,1,0,1,2,1,1,0,1,1,1,13,11,1,7,74


In [85]:
print(df.columns)

Index(['ID', 'female', 'nonwhite', 'unemplmt', 'primsev', 'Raceg4', 'B2a_0',
       'State', 'City', 'zipcode',
       ...
       'homeless_0_cd', 'S6_cd', 'GCS_0_cd', 'gcsg_0_cd', 'ncar_cd',
       'LRI7_0_cd', 'SRI7_0_cd', 'loc_cd', 'txtypeg_cd', 'ada_0_cd'],
      dtype='object', length=108)


In [86]:
# keep imputed cols, remove txtypeg_cd, remove continuous vars

cols = ['engage30', 'init']
continuous_vars = ['TRI_0_cd','TMI_0_cd','SES_0_cd','GVS_cd','DSS9_0_cd','LRI7_0_cd','SRI7_0_cd','GCS_0_cd']
for col in df.columns: # add features
    if '_cd' in col and col != 'txtypeg_cd' and col not in continuous_vars:
        cols.append(col)
df = df[cols]
df.shape

(23961, 32)

In [87]:
df.columns

Index(['engage30', 'init', 'female_cd', 'nonwhite_cd', 'unemplmt_cd',
       'primsev_cd', 'Raceg4_cd', 'prsatx_cd', 'TRIg_0_cd', 'TMIg_0_cd',
       'SESg_0_cd', 'gvsg_cd', 'tsd_0_cd', 'und15_cd', 'CWSg_0_cd',
       'srprobg_cd', 'S9y10_cd', 'dldiag_cd', 'dssg_0_cd', 'epsg_0_cd',
       'adhdg_0_cd', 'cdsg_0_cd', 'suicprbs_0_cd', 'cjsig_0_cd', 'lrig_0_cd',
       'srig_0_cd', 'homeless_0_cd', 'S6_cd', 'gcsg_0_cd', 'ncar_cd', 'loc_cd',
       'ada_0_cd'],
      dtype='object')

In [88]:
df['Raceg4_cd'].value_counts()

1    8902
3    7219
4    4097
2    3743
Name: Raceg4_cd, dtype: int64

In [89]:
# one-hot encode Raceg4_cd

one_hot = pd.get_dummies(df['Raceg4_cd'], prefix='Raceg4_cd')
df = df.drop('Raceg4_cd', axis = 1)
df = df.join(one_hot)

df.columns

Index(['engage30', 'init', 'female_cd', 'nonwhite_cd', 'unemplmt_cd',
       'primsev_cd', 'prsatx_cd', 'TRIg_0_cd', 'TMIg_0_cd', 'SESg_0_cd',
       'gvsg_cd', 'tsd_0_cd', 'und15_cd', 'CWSg_0_cd', 'srprobg_cd',
       'S9y10_cd', 'dldiag_cd', 'dssg_0_cd', 'epsg_0_cd', 'adhdg_0_cd',
       'cdsg_0_cd', 'suicprbs_0_cd', 'cjsig_0_cd', 'lrig_0_cd', 'srig_0_cd',
       'homeless_0_cd', 'S6_cd', 'gcsg_0_cd', 'ncar_cd', 'loc_cd', 'ada_0_cd',
       'Raceg4_cd_1', 'Raceg4_cd_2', 'Raceg4_cd_3', 'Raceg4_cd_4'],
      dtype='object')

In [90]:
df['loc_cd'].value_counts()

4     15667
5      3070
1      1604
2       890
8       789
7       694
9       653
99      431
6       148
3        15
Name: loc_cd, dtype: int64

In [91]:
# drop rows where loc_cd == 99
df = df[df.loc_cd != 99]

# group loc_cd based on Jordan's recommendation
df['loc_cd'] = np.where(df['loc_cd'].between(3,5), 3, df['loc_cd'])
df['loc_cd'] = np.where(df['loc_cd'].between(6,8), 6, df['loc_cd'])
print(df['loc_cd'].value_counts())

# one-hot encode loc_cd
one_hot = pd.get_dummies(df['loc_cd'], prefix='loc_cd')
df = df.drop('loc_cd', axis = 1)
df = df.join(one_hot)

df.columns

3    18752
6     1631
1     1604
2      890
9      653
Name: loc_cd, dtype: int64


Index(['engage30', 'init', 'female_cd', 'nonwhite_cd', 'unemplmt_cd',
       'primsev_cd', 'prsatx_cd', 'TRIg_0_cd', 'TMIg_0_cd', 'SESg_0_cd',
       'gvsg_cd', 'tsd_0_cd', 'und15_cd', 'CWSg_0_cd', 'srprobg_cd',
       'S9y10_cd', 'dldiag_cd', 'dssg_0_cd', 'epsg_0_cd', 'adhdg_0_cd',
       'cdsg_0_cd', 'suicprbs_0_cd', 'cjsig_0_cd', 'lrig_0_cd', 'srig_0_cd',
       'homeless_0_cd', 'S6_cd', 'gcsg_0_cd', 'ncar_cd', 'ada_0_cd',
       'Raceg4_cd_1', 'Raceg4_cd_2', 'Raceg4_cd_3', 'Raceg4_cd_4', 'loc_cd_1',
       'loc_cd_2', 'loc_cd_3', 'loc_cd_6', 'loc_cd_9'],
      dtype='object')

In [92]:
# drop rows not pertaining to opioids, amphetamines, or cocaine

opioids_mask = df['primsev_cd'] == 5
amphetamines_mask = df['primsev_cd'] == 2
cocaine_mask = df['primsev_cd'] == 4
df = df[amphetamines_mask | cocaine_mask | opioids_mask]
df.shape

(5458, 39)

In [93]:
df.to_csv("data_cleaned.csv")

In [94]:
total_seconds = int(time.time() - start_time)
hours = total_seconds // (60 * 60)
minutes = (total_seconds - hours*60) // 60
seconds = (total_seconds - hours*60) % 60
print("--- " + str(minutes) + " minutes " + str(seconds) + " seconds ---")

--- 0 minutes 2 seconds ---
